# Connecting to Recombee using Recombee Client API

In [11]:
#Python Recombee
from recombee_api_client.api_client import RecombeeClient
from recombee_api_client.api_requests import *
import json

client = RecombeeClient('ewa-sem-proj-dev', 'gZnZYXh3tk1QGAp4vinf4P6wpI6pA0P4xYkEGMiGnRtBWtUa7UHZesIcOJHBdSbp')
#print(client);

# Connecting to MySQL database to get the data and save it as JSON. For this Pandas has been used.

In [12]:
import json
import pymysql
import pandas

conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', password='root', database='SocialRecommendation')
cursor = conn.cursor()

# Quering data for getting the revies data for recommendation
query = 'select distinct(userid) as _userid, REPLACE(LTRIM(RTRIM(name)), SPACE(1),"_") as _name, reviewrating as _reviewrating  from review'

results = pandas.read_sql_query(query, conn)
results.to_json("data_train.json",orient='records')

# Getting the list of distinct users from review table.
query_users='select distinct(userid) as _userid from review'

results = pandas.read_sql_query(query_users, conn)
results.to_json("data_train_users.json",orient='records')

In [3]:
#client.send(DeleteItem('SamsungQLED'))
#client.send(AddItemProperty('_userid','string'))
#client.send(AddItemProperty('_name','string'))
#client.send(AddItemProperty('_reviewrating','string'))

'ok'

# Inserting data to Recombee server that will be used further for recommendation.

In [13]:
requests = []
with open('data_train.json') as f:
    data = json.loads(f.read())
    for item in data:
        print("Data added to request - "+ item['_name'])
        r = SetItemValues(item['_name'], item, cascade_create=True)
        requests.append(r)

res = client.send(Batch(requests))
print("Data successfully pushed to Recombee server.")

Data added to request - Millennium_Park
Data added to request - India_House_Restaurant_-_Chicago
Data added to request - Bombay_Wraps
Data added to request - Lickity_Split_Frozen_Custard
Data added to request - Taste_of_Peru
Data added to request - Pita_Inn
Data added to request - Captains_Walk_Shopping_Mall
Data added to request - Skokie_Fashion_Square
Data added to request - Jewel-Osco
Data added to request - Parivar_Grocers
Data added to request - Patel_Brothers
Data added to request - Shedd_Aquarium
Data added to request - Bar_Louie_-_Naperville
Data added to request - Pub_56
Data added to request - Quigley
Data added to request - Stateway_Park
Data added to request - Chicago_Curry_House_Restaurant
Data added to request - Nepal_House_Indian_Nepalese_Restaurant
Data successfully pushed to Recombee server.


# Performing recommendation for each user based on the data that was pushed and the recommended data is being stored as csv file.

In [14]:

import csv

out =open('output_recombee.csv','w',newline='')
output = csv.writer(out)
name =''

# Fetching the list of users from Json File
with open('data_train_users.json') as g:
    users = json.loads(g.read())
    for user in users:
        #print(user['_userid'])
        # For each user,perform the recommendation
        with open('data_train.json') as f:
            interactions = json.loads(f.read())   
            for item in interactions:
                if(user['_userid']==item['_userid']):
                    recommended = client.send(RecommendItemsToItem(item['_name'], item['_userid'], 10, cascade_create=True))
                    name='['
                    end=']'
                    for re in recommended['recomms']:
                        name += '\''+re['id']+'\','
                    name+=end
                    name=name.replace(",]","]")
                    name=name.replace("_"," ")
                    print(item['_userid'],name)
                    output.writerow([item['_userid'],name])
                    break;
out.close()
        

user1 ['Bombay Wraps','India House Restaurant - Chicago','Nepal House Indian Nepalese Restaurant','Pita Inn','Parivar Grocers','Patel Brothers','Chicago Curry House Restaurant','Quigley','Shedd Aquarium','Pub 56']
user2 ['Pub 56','Taste of Peru','Nepal House Indian Nepalese Restaurant','Parivar Grocers','Millennium Park','Bar Louie - Naperville','Stateway Park','Patel Brothers','Quigley','Bombay Wraps']
user3 ['Captains Walk Shopping Mall','Skokie Fashion Square','Nepal House Indian Nepalese Restaurant','Parivar Grocers','Millennium Park','Patel Brothers','Chicago Curry House Restaurant','Quigley','Bombay Wraps','Shedd Aquarium']
user4 ['Parivar Grocers','Patel Brothers','Stateway Park','Captains Walk Shopping Mall','Skokie Fashion Square','Taste of Peru','Bar Louie - Naperville','India House Restaurant - Chicago','Pub 56','Nepal House Indian Nepalese Restaurant']
user5 ['Quigley','Pub 56','Bar Louie - Naperville','Nepal House Indian Nepalese Restaurant','Pita Inn','Parivar Grocers','M